In [25]:
import pandas as pd
import numpy as np

In [117]:
x_train = pd.read_csv('data/AMF_train_X.csv')
y_train = pd.read_csv('data/AMF_train_Y.csv')

In [118]:
x_train

,Index,Share,Day,Trader,OTR,OCR,OMR,min_time_two_events,mean_time_two_events,10_p_time_two_events,...,min_dt_TV1_TV2,mean_dt_TV1_TV2,med_dt_TV1_TV2,min_dt_TV1_TV3,mean_dt_TV1_TV3,med_dt_TV1_TV3,min_dt_TV1_TV4,mean_dt_TV1_TV4,med_dt_TV1_TV4,NbSecondWithAtLeatOneTrade
0,1,Isin_8,Date_24,Trader_10,2.272727,8.333333,12.500000,0.0,5117.830300,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,2,Isin_87,Date_29,Trader_10,1.696629,25.166667,21.571429,0.0,1846.968401,0.000074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
2,3,Isin_87,Date_23,Trader_10,1.482759,47.300000,118.250000,0.0,686.300630,0.000071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63
3,4,Isin_12,Date_11,Trader_10,1.705882,14.500000,29.000000,0.0,2174.335265,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,5,Isin_87,Date_9,Trader_10,1.517730,26.750000,NaN,0.0,944.008551,0.000071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105777,105778,Isin_84,Date_27,Trader_60,1.150000,23.000000,NaN,0.0,736.500080,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
105778,105779,Isin_102,Date_18,Trader_60,1.352941,23.000000,NaN,0.0,63.672695,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
105779,105780,Isin_12,Date_13,Trader_60,1.146667,86.000000,NaN,0.0,708.440816,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16
105780,105781,Isin_27,Date_30,Trader_60,1.179191,204.000000,NaN,0.0,384.893976,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35


In [16]:
y_train

,Trader,type
0,Trader_285,MIX
1,Trader_114,NON HFT
2,Trader_110,NON HFT
3,Trader_57,NON HFT
4,Trader_128,NON HFT
...,...,...
81,Trader_140,MIX
82,Trader_451,MIX
83,Trader_438,MIX
84,Trader_278,HFT


In [18]:
n_rows, n_cols = x_train.shape

### The NaN ratio in each columns

In [21]:
x_train.isnull().sum()/n_rows

Index                         0.000000
Share                         0.000000
Day                           0.000000
Trader                        0.000000
OTR                           0.015910
OCR                           0.000000
OMR                           0.450095
min_time_two_events           0.000000
mean_time_two_events          0.000000
10_p_time_two_events          0.000000
med_time_two_events           0.000000
25_p_time_two_events          0.000000
75_p_time_two_events          0.000000
90_p_time_two_events          0.000000
max_time_two_events           0.000000
min_lifetime_cancel           0.000000
mean_lifetime_cancel          0.000000
10_p_lifetime_cancel          0.000000
med_lifetime_cancel           0.000000
25_p_lifetime_cancel          0.000000
75_p_lifetime_cancel          0.000000
90_p_lifetime_cancel          0.000000
max_lifetime_cancel           0.000000
NbTradeVenueMic               0.000000
MaxNbTradesBySecond           0.000000
MeanNbTradesBySecond     

In [120]:
x_train_data = np.array(x_train)[:,4:]

In [30]:
ind2type = {0: 'HFT', 1: 'MIX', 2: 'NON HFT'}
type2ind = {'HFT': 0, 'MIX': 1, 'NON HFT': 2}

In [31]:
y_train

,Trader,type
0,Trader_285,MIX
1,Trader_114,NON HFT
2,Trader_110,NON HFT
3,Trader_57,NON HFT
4,Trader_128,NON HFT
...,...,...
81,Trader_140,MIX
82,Trader_451,MIX
83,Trader_438,MIX
84,Trader_278,HFT


In [44]:
name2type = dict(zip(y_train.Trader, y_train.type))

In [119]:
x_train = x_train.fillna(-1)

In [75]:
x_train['label'] = x_train.apply(lambda row: 0 if name2type[row.Trader] == 'HFT' else (1 if name2type[row.Trader] == 'MIX' else 2), axis=1)

In [76]:
y_train_data = x_train['label']

In [77]:
from sklearn.model_selection import train_test_split

In [121]:
x_train_b, x_test_b, y_train_b, y_test_b = train_test_split(x_train_data, y_train_data, test_size=0.3)

In [122]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC

# Create the SVM
svm = LinearSVC(random_state=42)

# Make it an Multilabel classifier

# Fit the data to the Multilabel classifier
svm.fit(x_train_b, y_train_b)

# Get predictions for test data
y_test_pred = svm.predict(x_test_b)

In [123]:
len(np.where(y_test_pred == y_test_b)[0]) / len(y_test_b)

0.4761934772333386

In [124]:
x_real_test = pd.read_csv('data/AMF_test_X.csv')

In [125]:
x_real_test_data = np.array(x_real_test.fillna(-1))[:,4:]

In [126]:
y_test_predict = svm.predict(x_real_test_data)

In [127]:
x_real_test_data.shape

(85304, 35)

In [128]:
x_train_data.shape

(105782, 35)

In [129]:
y_test_predict


array([2, 2, 2, ..., 1, 1, 1], dtype=int64)

In [131]:
res = [(row.Trader, y_test_predict[i])  for i, row in x_real_test.iterrows()]
    

In [133]:
dicts = {}
for row in res:
    if row[0] in dicts:
        dicts[row[0]].append(row[1])
    else:
        dicts[row[0]] = [row[1]]

In [142]:
final = {}
for k,v in dicts.items():
    if len(np.where(v == 0)[0])/len(v) > 0.7:
        final[k] = 'HFT'
    elif len(np.where(v == 0)[0])/len(v) > 0.3:
        final[k] = 'MIX'
    else:
        final[k] = 'NON HFT'

In [150]:
import csv
(pd.DataFrame.from_dict(data=final, orient='index').to_csv('dict_file.csv'))